In [95]:
import cv2
import numpy as np
import os
import json

path_img = 'E:/python/projects/scp3/Task/dataset/image'
path_mask = 'E:/python/projects/scp3/Task/dataset/human_parsing'
path_pose = 'E:/python/projects/scp3/Task/dataset/pose_json'

directory_img = os.fsencode(path_img)
directory_mask = os.fsencode(path_mask)
directory_pose = os.fsencode(path_pose)
files = 0
for i in os.listdir(directory_img):
    for m in os.listdir(directory_mask):
        for p in os.listdir(directory_pose):
            
            filename_img = os.fsdecode(i)[:-7]
            filename_mask = os.fsdecode(m)[:-7]
            filename_pose = os.fsdecode(p)[:-18]
            if filename_img != '.D':
                if filename_img == filename_mask == filename_pose:

                    mask = cv2.imread(f'E:/python/projects/scp3/Task/dataset/human_parsing/{filename_mask}_00.png')
                    with open(f'E:/python/projects/scp3/Task/dataset/pose_json/{filename_pose}_00_keypoints.json') as json_file:
                        data = json.load(json_file)

                    shoulders = []
                    for d in data['people']:
                        for n in d['pose_keypoints_2d'][3:5]:
                            shoulders.append(int(round(n)))

                    waist = []
                    for d in data['people']:
                        for n in d['pose_keypoints_2d'][24:26]:
                            waist.append(int(round(n)))

                    left_11 = []
                    for d in data['people']:
                        for n in d['pose_keypoints_2d'][33:35]:
                            left_11.append(int(round(n)))


                    right_14 = []
                    for d in data['people']:
                        for n in d['pose_keypoints_2d'][42:44]:
                            right_14.append(int(round(n)))

                    mask[np.where((mask==[0,0,0]).all(axis=2))] = [255,255,255]
                    waist_middle = int(round((waist[1] - ((waist[1] - shoulders[1])/1.7)), 0))

                    purple = [128,0,64]
                    blue = [128,128,0] 
                    red = [0,0,192]

                    p_Y, p_X = np.where(np.all(mask==purple,axis=2))
                    b_Y, b_X = np.where(np.all(mask==blue,axis=2))      
                    r_Y, r_X = np.where(np.all(mask==red,axis=2))
                    try:
                        zipped = np.column_stack((b_X,b_Y))

                        b_zipped=[]
                        for b in zipped:
                            if b[1] > waist[1]:
                                b_zipped.append(b)

                        for k in b_zipped:
                            mask[k[1],k[0]] = [0,0,0]
                    except:
                        pass

                    mask[np.where((mask==[128,0,64]).all(axis=2))] = [0,0,0]
                    mask[np.where((mask==[0,0,192]).all(axis=2))] = [0,0,0]
                    mask[np.where((mask==[0,64,128]).all(axis=2))] = [0,0,0]
                    mask[np.where((mask==[0,64,0]).all(axis=2))] = [0,0,0]    

                    if shoulders[0] != 0:
                        mask[0:waist_middle, 0:768] = [255,255,255]
                    elif shoulders[0] == 0:
                        mask[np.where((mask==[128,128,0]).all(axis=2))] = [0,0,0]    
                    if left_11[1] > right_14[1]:
                        mask[left_11[1]:1024, 0:768] = [255,255,255]
                    elif right_14[1] > left_11[1]:
                        mask[right_14[1]:1024, 0:768] = [255,255,255]

                    image = cv2.imread(f'E:/python/projects/scp3/Task/dataset/image/{filename_img}_00.jpg')
                    gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
                    masked = cv2.bitwise_or(image, image, mask=gray)
                    masked[np.where((masked==[0,0,0]).all(axis=2))] = [128,128,128]
                    cv2.imwrite(f'E:/python/projects/scp3/Task/dataset/masked/{filename_img}_00_masked.jpg', masked)

В таком виде алгоритм работает достаточно медленно, тк приходится перебирать файлы из трех папок. Я бы сделала раздельно: сначала переделываем маску, потом применяем ее. В задании, однако, было сказано "написать алгоритм", так что я соединила все вместе.